## Задание: обучите модель классификации букв для задачи расстановки ударения с помощью методов из библиотеки transformers.
## Датасет для обучения можно взять отсюда: https://github.com/Koziev/NLP_Datasets/blob/master/Stress/all_accents.zip

### 1. Напишите класс для Dataset/Dataloder и разбейте данные на случайные train / test сплиты в соотношении 50:50. (1 балл)
### 2. Попробуйте обучить одну или несколько из моделей: Bert, Albert, Deberta. Посчитайте метрику Accuracy на train и test. (1 балл). При преодолении порога в Accuracy на test 0.8: (+1 балл), 0.85: (+2 балла), 0.89: (+3 балла).
### Пример конфигурации для deberta: https://huggingface.co/IlyaGusev/ru-word-stress-transformer/blob/main/config.json

In [2]:
!git clone https://github.com/KuzmaKhrabrov/character-tokenizer.git

Cloning into 'character-tokenizer'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 20 (delta 5), reused 10 (delta 3), pack-reused 0
Receiving objects: 100% (20/20), 5.89 KiB | 1.96 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [3]:
!pip install transformers

In [4]:
import string
import sys
sys.path.append("/kaggle/working/character-tokenizer")
from charactertokenizer import CharacterTokenizer

device = 'cuda'


chars = "АаБбВвГгДдЕеЁёЖжЗзИиЙйКкЛлМмНнОоПпРрСсТтУуФфХхЦцЧчШшЩщЪъЫыЬьЭэЮюЯя"
model_max_length = 64
tokenizer = CharacterTokenizer(chars, model_max_length)

In [5]:
example = "Привет"
tokens = tokenizer(example)
print(tokens)
print(tokenizer.decode(tokens['input_ids']))

{'input_ids': [0, 39, 42, 26, 12, 18, 46, 1], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[CLS]Привет[SEP]


## Импортируем датасет

In [6]:
import pandas as pd
data = pd.read_csv('/kaggle/input/accents/all_accents.tsv', sep='\t')
data

,-де,-д^е
0,-ка,-к^а
1,-либо,-л^ибо
2,-нибудь,-ниб^удь
3,-с,-с
4,-таки,-так^и
...,...,...
1680529,ӂюль-верновский,ӂюль-в^ерновский
1680530,ӂюрить,ӂюр^ить
1680531,ӂӂение,ӂӂ^ение
1680532,ӂӂенный,ӂӂенный


In [7]:
data.rename(columns={"-де": "words", "-д^е": "answers"}, inplace=True)

In [8]:
data

,words,answers
0,-ка,-к^а
1,-либо,-л^ибо
2,-нибудь,-ниб^удь
3,-с,-с
4,-таки,-так^и
...,...,...
1680529,ӂюль-верновский,ӂюль-в^ерновский
1680530,ӂюрить,ӂюр^ить
1680531,ӂӂение,ӂӂ^ение
1680532,ӂӂенный,ӂӂенный


In [9]:
data.shape

(1680534, 2)

In [10]:
# проверка, все ли строки валидные

data[(data['answers'].str.contains('^')==True)].shape

(1680534, 2)

In [11]:
from sklearn.utils import shuffle
data = shuffle(data)

In [12]:
def get_emphasis_number(word: str):
    vowel_list = 'еуыаоэюия'
    order = 0
    for idx, letter in enumerate(word):
        if letter in vowel_list:
            order += 1
        
        if '^' not in word[idx:]:
            if (order == 0):
                return 1
            return order
        
    
print(get_emphasis_number('ӂӂ^ение'))
print(get_emphasis_number('ӂӂени^е'))

1
3


In [13]:
data["labels"] = data["answers"].apply(get_emphasis_number)

In [14]:
data.labels.unique()

array([ 4,  3,  2,  5,  1,  6,  7,  9,  8, 13, 10, 11, 12, 15, 21])

In [15]:
data.labels.value_counts()

labels
3     586477
2     563967
1     234078
4     209830
5      64004
6      16992
7       4056
8        900
9        171
10        38
11        12
13         4
12         3
15         1
21         1
Name: count, dtype: int64

In [16]:
print(data[data['labels']==21].words)
print(data[data['labels']==15].words)

624999    лланвайрпуллгуингиллгогерихуирндробуллллантиси...
Name: words, dtype: object
790570    никотинамидадениндинуклеотидфосфат
Name: words, dtype: object


In [17]:
data = data[data['labels']<9]

In [18]:
number_of_classes = len(data.labels.unique())
print(number_of_classes)

8


In [19]:
new_data = data.copy()

In [20]:
classes = []
n_samples = 4000

for label in range(1, number_of_classes + 1):
    classes.append(new_data[new_data['labels']==label])
    if len(classes[-1]) > n_samples:
        classes[-1] = classes[-1].sample(n=n_samples)

In [21]:
for cl in classes:
    print(cl.shape)

(4000, 3)
(4000, 3)
(4000, 3)
(4000, 3)
(4000, 3)
(4000, 3)
(4000, 3)
(900, 3)


In [22]:
prepared_data = pd.concat(classes)

In [23]:
prepared_data.labels.value_counts()

labels
1    4000
2    4000
3    4000
4    4000
5    4000
6    4000
7    4000
8     900
Name: count, dtype: int64

In [24]:
prepared_data.reset_index(drop=True, inplace=True)

## Напишем кастомный датасет

In [25]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split

In [26]:
class CustomDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
#         elem = self.words.iloc[idx]
#         word = self.tokenizer(elem["words"], padding=True, return_tensors='pt')
# #         ans = self.tokenizer(elem["answers"])
#         label = elem["label"]
#         return {"word": word, "label": label}
        return (self.x[idx], self.y[idx])
    
def data_collator(batch):
#     y = torch.Tensor([p[1] for p in batch]).to(model.device)
#     print("Before")
#     print("Batch:", batch)
    y = [torch.zeros(number_of_classes) for i in range(len(batch))]
    for vec, p in zip(y, batch):
        label = p[1]
        vec[p[1] - 1] = 1
    y = torch.stack(y, dim=0).to(device)
        
#     print("After")
#     x = tokenizer([p[0] for p in batch], return_tensors='pt', padding=True).to(model.device)
#     x = tokenizer([p['word']['input_ids'] for p in batch], return_tensors='pt', padding=True)
#     x = [p['word'] for p in batch]
    x = tokenizer([p[0] for p in batch], return_tensors='pt', padding=True).to(device)
    return (x, torch.Tensor(y))
        

In [27]:
dataset = CustomDataset(prepared_data.words.values, prepared_data.labels)
train_size = int(len(dataset) * 0.5)
test_size = int(len(dataset) - train_size)
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
val_size = int(len(train_dataset) * 0.1)
train_dataset, val_dataset = random_split(train_dataset, [len(train_dataset) - val_size, val_size])

In [28]:
len(train_dataset)

13005

In [29]:
BATCH_SIZE = 128
N = 0

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,num_workers=N,
                          collate_fn=data_collator, drop_last = True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=N, 
                          collate_fn=data_collator, drop_last = True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=N,
                         collate_fn=data_collator, drop_last = False)

In [30]:
batch = next(iter(train_loader))

In [31]:
batch[0][:2]

{'input_ids': tensor([[ 0, 40,  8, 42, 46, 14, 38, 44, 36, 38, 34, 18, 36, 30, 32,  8, 46, 48,
          42,  8,  1,  4,  4,  4,  4],
         [ 0, 40, 42, 38, 14, 48, 16, 26,  1,  4,  4,  4,  4,  4,  4,  4,  4,  4,
           4,  4,  4,  4,  4,  4,  4]], device='cuda:0'),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0]], device='cuda:0'),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
          0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0]], device='cuda:0')}

In [32]:
batch[1][:2]

tensor([[0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0.]], device='cuda:0')

## Тренировочный цикл

In [33]:
from tqdm import tqdm
import torch
from torch.optim.lr_scheduler import StepLR
from torch.nn.functional import softmax

SAVE_PATH = "model_params"


def train_loop(
    model, train_dataloader, val_dataloader, 
    max_epochs=10, 
    lr=1e-5,
    eval_steps = 10000
):
    optimizer = torch.optim.Adam(params = model.parameters(), lr=lr)
    scheduler = StepLR(optimizer, step_size = 3, gamma=0.5)
    best_f1 = float('-inf')
    
    for epoch in range(max_epochs):
        print('EPOCH', epoch)
        losses = list()
        for i, (x, y) in tqdm(enumerate(train_dataloader)):
#             inputs = [elem['input_ids'] for elem in x]
#             attention_mask = [elem['attention_mask'] for elem in x]
#             token_type_ids = [elem['token_type_ids'] for elem in x]
            output = model(
                input_ids=x.input_ids,
                attention_mask=x.attention_mask,
                token_type_ids=x.token_type_ids,
                labels=y,
                return_dict=True
            )
#             print("labels", labels)
#             print("output", output)
            loss = output.loss
            
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
            losses.append(loss.item())
            
            if i % eval_steps == 0:
                model.eval()
                train_loss = np.mean(losses[-eval_steps:])
                eval_loss, eval_f1, eval_accuracy, _ = evaluate_model(model, val_dataloader)
                if eval_f1 > best_f1:
                    best_f1 = eval_f1
                    torch.save(model.state_dict(), SAVE_PATH)
                print(f'step {i} train_loss: {train_loss:.3} eval_loss: {eval_loss:.3} eval_f1: {eval_f1:.3} eval_accuracy: {eval_accuracy:.3}')
                model.train()
        scheduler.step()

In [34]:
from sklearn.metrics import f1_score, accuracy_score


def evaluate_model(model, test_dataloader):
    num = 0
    den = 0
    y_true = list()
    y_pred = list()
    y_pred_prob = list()
    f1_valid = .0
    for x, y in test_dataloader:
        with torch.no_grad():
            output = model(
                input_ids=x.input_ids,
                attention_mask=x.attention_mask,
                token_type_ids=x.token_type_ids,
                labels=y,
                return_dict=True
            )
            loss = output.loss
            
            num += len(x) * loss.item()
            den += len(x)
            
            y_pred.extend(torch.argmax(output.logits, 1).tolist())
            y_pred_prob.extend(softmax(output.logits, dim = 1)[:, 1].tolist())
            y_true.extend(torch.argmax(y, 1).tolist())
            
    val_loss = num / den
    f1_valid = f1_score(y_true, y_pred, average = 'micro')
    accuracy_valid = accuracy_score(y_true, y_pred)
    return val_loss, f1_valid, accuracy_valid, y_pred_prob

## Подгрузим модель

In [35]:
from transformers import BertForSequenceClassification, DebertaV2ForTokenClassification, DebertaV2Config


# encoder =  BertGenerationEncoder.from_pretrained("bert-large-uncased", bos_token_id=0, eos_token_id=1)
# decoder = BertGenerationDecoder.from_pretrained("bert-l-uncased", add_cross_attention=True, is_decoder=True,
#                                                bos_token_id=0, eos_token_id=1)

# configuration = DebertaV2Config()

model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-uncased", problem_type="multi_label_classification", num_labels=number_of_classes).cuda()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Цикл обучения

In [36]:
EPOCHS = 40

train_loop(model, train_loader, val_loader, max_epochs=EPOCHS, lr=8e-5, eval_steps=40)

EPOCH 0


1it [00:04,  4.68s/it]

step 0 train_loss: 0.674 eval_loss: 0.617 eval_f1: 0.0831 eval_accuracy: 0.0831


41it [00:20,  1.17s/it]

step 40 train_loss: 0.379 eval_loss: 0.319 eval_f1: 0.262 eval_accuracy: 0.262


...

41it [00:16,  1.34it/s]

step 40 train_loss: 0.113 eval_loss: 0.125 eval_f1: 0.795 eval_accuracy: 0.795


81it [00:30,  1.42it/s]

step 80 train_loss: 0.113 eval_loss: 0.125 eval_f1: 0.796 eval_accuracy: 0.796


101it [00:37,  2.68it/s]


In [37]:
num = 0
den = 0
y_true = list()
y_pred = list()
y_pred_prob = list()
f1_valid = .0


for x, y in val_loader:
        with torch.no_grad():
            output = model(
                input_ids=x.input_ids,
                attention_mask=x.attention_mask,
                token_type_ids=x.token_type_ids,
                labels=y,
                return_dict=True
            )
            loss = output.loss
            
            y_pred.extend(torch.argmax(output.logits, 1).tolist())
            y_pred_prob.extend(softmax(output.logits, dim = 1)[:, 1].tolist())
            y_true.extend(torch.argmax(y, 1).tolist())
            
            print(y_pred)
            print(y_true)
            break

[6, 5, 5, 3, 2, 3, 2, 3, 5, 0, 3, 6, 6, 3, 4, 6, 3, 6, 2, 2, 3, 3, 3, 6, 1, 5, 6, 5, 4, 5, 5, 1, 5, 4, 2, 4, 0, 4, 3, 0, 2, 5, 5, 0, 6, 1, 6, 6, 3, 1, 0, 5, 4, 2, 0, 3, 5, 1, 7, 2, 3, 6, 1, 6, 5, 4, 1, 6, 5, 4, 2, 6, 3, 6, 2, 6, 1, 6, 1, 6, 6, 5, 4, 0, 4, 5, 3, 1, 5, 0, 4, 1, 0, 4, 4, 6, 4, 5, 4, 6, 6, 2, 5, 3, 2, 6, 5, 4, 7, 5, 5, 2, 4, 1, 3, 2, 0, 5, 3, 4, 6, 6, 2, 1, 3, 2, 4, 1]
[6, 4, 5, 3, 2, 3, 2, 3, 5, 0, 3, 6, 6, 3, 4, 6, 2, 6, 2, 2, 3, 2, 3, 6, 1, 5, 5, 5, 4, 5, 5, 1, 5, 4, 2, 4, 2, 4, 3, 0, 2, 5, 5, 0, 6, 1, 6, 5, 3, 0, 0, 5, 5, 2, 3, 3, 5, 0, 7, 1, 3, 6, 1, 6, 5, 4, 1, 4, 5, 4, 2, 6, 4, 6, 2, 6, 1, 6, 2, 6, 6, 5, 4, 1, 4, 5, 3, 2, 5, 0, 4, 2, 1, 4, 4, 6, 4, 5, 4, 6, 6, 2, 4, 3, 2, 6, 5, 4, 7, 5, 5, 1, 4, 1, 2, 1, 0, 5, 3, 4, 6, 6, 2, 2, 3, 1, 4, 0]


# Проверка на тестовой выборке

In [38]:
y_true = list()
y_pred = list()
y_pred_prob = list()
f1_valid = .0


for x, y in val_loader:
        with torch.no_grad():
            output = model(
                input_ids=x.input_ids,
                attention_mask=x.attention_mask,
                token_type_ids=x.token_type_ids,
                labels=y,
                return_dict=True
            )
            loss = output.loss
            
            y_pred.extend(torch.argmax(output.logits, 1).tolist())
            y_pred_prob.extend(softmax(output.logits, dim = 1)[:, 1].tolist())
            y_true.extend(torch.argmax(y, 1).tolist())
            
accuracy_test = accuracy_score(y_true, y_pred)
            

In [39]:
accuracy_test

0.7904829545454546

# Если загрузить веса с лучшей эпохи

In [45]:
model_best = model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-uncased", problem_type="multi_label_classification", num_labels=number_of_classes).cuda()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [46]:
model_best.load_state_dict(torch.load(SAVE_PATH))

<All keys matched successfully>

In [48]:
model_best.eval()


y_true = list()
y_pred = list()
y_pred_prob = list()
f1_valid = .0


for x, y in val_loader:
        with torch.no_grad():
            output = model(
                input_ids=x.input_ids,
                attention_mask=x.attention_mask,
                token_type_ids=x.token_type_ids,
                labels=y,
                return_dict=True
            )
            loss = output.loss
            
            y_pred.extend(torch.argmax(output.logits, 1).tolist())
            y_pred_prob.extend(softmax(output.logits, dim = 1)[:, 1].tolist())
            y_true.extend(torch.argmax(y, 1).tolist())
            
accuracy_test = accuracy_score(y_true, y_pred)
            

In [49]:
accuracy_test

0.8082386363636364

# Итог: с помощью BERT получилось добиться accuracy = 0.808